<a href="https://colab.research.google.com/github/JhonaGo/BI-Tools/blob/main/Multiple_TimeSeries_Forecast_StoreSku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FORECASTING

Paqueterias

In [6]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install -U "gluonts[torch]==0.11.0" "optuna~=2.10.0"
!pip install ipykernel
!pip install --upgrade nbformat
!pip install lightgbm xgboost
!pip install seaborn
!pip install distributed
!pip install datasetsforecast
!pip install darts
!pip install mlforecast
!pip install statsforecast

  Using cached gluonts-0.11.0-py3-none-any.whl.metadata (9.6 kB)
Requested gluonts==0.11.0 from https://files.pythonhosted.org/packages/62/ec/6ff7fb405af0271a056ac86355a660cc8b450116036f67d8227922cc244a/gluonts-0.11.0-py3-none-any.whl (from gluonts[torch]==0.11.0) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    rpy2 (<3.*,>=2.9.*) ; extra == 'r'
          ~~~^
Please use pip<24.1 if you need to use this version.
ERROR: Could not find a version that satisfies the requirement gluonts==0.11.0 (from versions: 0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 0.6.1, 0.6.2, 0.6.3, 0.6.4, 0.6.5, 0.6.6, 0.6.7, 0.6.8, 0.6.9, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.9.5, 0.9.6, 0.9.7, 0.9.8, 0.9.9, 0.9.10, 0.10.0rc1, 0.10.0, 0.10.1, 0.10.2, 0.10.3, 0.10.4, 0.10.5, 0.10.6, 0.10.7, 0.10.8, 0.10.9, 0.10.10, 0.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Conjunto de datos

In [9]:
# data
data23 = pd.read_excel("/content/drive/MyDrive/Forecasting_PC&Tissue/TISSUE/23.xlsx",
                    sheet_name="Export")
data24 = pd.read_excel("/content/drive/MyDrive/Forecasting_PC&Tissue/TISSUE/24.xlsx",
                    sheet_name="Export")
data25 = pd.read_excel("/content/drive/MyDrive/Forecasting_PC&Tissue/TISSUE/25.xlsx",
                    sheet_name="Export")

# concatenate
data = pd.concat([data23, data24, data25], ignore_index=True)

data.head(5)

,Material[NomMaterial],Material[SKU],Material[Submarca],DIM LTTiempo[IdAnio],DIM LTTiempo[IdMes],Mercado[NomMercado],Material[Sector],Site[Site],Negocio[NomNegocio2],TRF_MAESTRO_DESTINATARIOS[region_nombre],...,[ING_COMERCIAL_OP_2],[GASTOC_CLIENTE_PPTO_2],[ING_COMERCIAL_PPTO_2],[VOL_Directorio],[ING_BRUTO_Directorio_2],[GASTO_CLIENTE_Directorio_2],[ING_COMERCIAL_Directorio_2],[VOL_ONE_PLAN],[VOL_PPTO],[UNIDAD_OP]
0,PA COTIDIAN PANTS G 8/10,AB30333182,PANTS,2023,1,Mercado Interno,PANALES ADULTOS,Mty,PERSONAL CARE,Nacional,...,884.350777,0.0,147.391796,NaN,NaN,NaN,NaN,0.093000,0.0155,1.162500
1,PA COTIDIAN PANTS G 8/10,AB30333182,PANTS,2023,2,Mercado Interno,PANALES ADULTOS,Mty,PERSONAL CARE,Nacional,...,884.311218,0.0,147.391796,NaN,NaN,NaN,NaN,0.092996,0.0155,1.162448
2,PA COTIDIAN PANTS G 8/10,AB30333182,PANTS,2023,3,Mercado Interno,PANALES ADULTOS,Mty,PERSONAL CARE,Nacional,...,884.350777,0.0,147.391796,NaN,NaN,NaN,NaN,0.093000,0.0155,1.162500
3,PA COTIDIAN PANTS G 8/10,AB30333182,PANTS,2023,4,Mercado Interno,PANALES ADULTOS,Mty,PERSONAL CARE,Nacional,...,NaN,0.0,147.391796,NaN,NaN,NaN,NaN,NaN,0.0155,NaN
4,PA COTIDIAN PANTS G 8/10,AB30333182,PANTS,2023,5,Mercado Interno,PANALES ADULTOS,Mty,PERSONAL CARE,Nacional,...,NaN,0.0,147.391796,NaN,NaN,NaN,NaN,NaN,0.0155,NaN


### New data frame

Catalogo de productos

In [10]:
# material catalogue
productos = pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Catalogo_Productos.xlsx",
                          sheet_name= "Catalogo")

# select columns
productos = productos.iloc[:, [8, 12]]

# rapid change of colums name
productos.columns = ["Material[SKU]", "nombre_unificado"]

productos.head(5)

,Material[SKU],nombre_unificado
0,AB11000855,Big Roll 400 24/4
1,AB10335148,Elite Facial Mega Diseños 48/180
2,AB10331522,Elite Neutro 18016.2M 60/32
3,AB10335179,Elite Facial Mega Spa 48/180
4,AB10335315,Elite Facial Mega Lav 48/180


Homologación de nombres de materiales

In [11]:
# merge dataset and material catalogue
data = pd.merge(data, productos, on="Material[SKU]", how="left")

# new info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50223 entries, 0 to 50222
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Material[NomMaterial]                      50215 non-null  object 
 1   Material[SKU]                              50215 non-null  object 
 2   Material[Submarca]                         50215 non-null  object 
 3   DIM LTTiempo[IdAnio]                       50223 non-null  int64  
 4   DIM LTTiempo[IdMes]                        50223 non-null  int64  
 5   Mercado[NomMercado]                        50223 non-null  object 
 6   Material[Sector]                           50215 non-null  object 
 7   Site[Site]                                 50223 non-null  object 
 8   Negocio[NomNegocio2]                       50223 non-null  object 
 9   TRF_MAESTRO_DESTINATARIOS[region_nombre]   50223 non-null  object 
 10  TRF_MAESTRO_DESTINATAR

Filtro de criterios en el dataset

In [12]:
# list of regions
print(data['TRF_MAESTRO_DESTINATARIOS[region_nombre]'].unique())

# filter
data = data[data['TRF_MAESTRO_DESTINATARIOS[region_nombre]'] == 'Zona Norte']

['Nacional' 'Sin Region' 'Zona Centro' 'Zona Centro 2' 'Zona Norte'
 'Zona Occidente' 'Zona Sur' 'Zona Valle de Mexico' ' Regional Sur']


In [13]:
# list of clientes in regions
data["TRF_MAESTRO_DESTINATARIOS[cliente_nombre]"].unique()

array(['LA MISION', 'MERCADOSCOAHUILENSES', 'UNPACO', 'ZAPETAS',
       'ERIK LOAIZA', 'PARRAL', 'GRAN D', 'MACONDGMX', 'H&D', 'DAPSA',
       'LINEAS EXCLUSIVAS', 'SUMERCA', 'COMERCIAL TREVIÑO',
       'ENRIQUE CANTU', 'FERNANDEZ MARKET', 'WULMEX', 'PICK-ONE',
       'ALDEKOM', 'JD ABARROTES', 'EMPAQUES DESECHABLES', 'FLORIDO',
       'ALONSO PACHECO GARCIA', 'EXPORTADORA TAMEZ'], dtype=object)

Nuevo conjunto de datos

In [14]:
data = data.iloc[:,[3,4,11,15,33]]
data.head(5)

,DIM LTTiempo[IdAnio],DIM LTTiempo[IdMes],TRF_MAESTRO_DESTINATARIOS[cliente_nombre],[VOL],nombre_unificado
7264,2023,1,LA MISION,10.00000,Mascarilla Tiquirurgica 20/5
7265,2023,1,MERCADOSCOAHUILENSES,0.89999,Mascarilla Tiquirurgica 20/5
7266,2023,1,UNPACO,6.00000,Mascarilla Tiquirurgica 20/5
7267,2023,1,ZAPETAS,NaN,Mascarilla Tiquirurgica 20/5
7268,2023,2,LA MISION,1.00000,Mascarilla Tiquirurgica 20/5


### Change data type

Fecha

In [15]:
# crete date
data["date"] = data['DIM LTTiempo[IdAnio]'].astype(str) + "-" + data['DIM LTTiempo[IdMes]'].astype(str)

# change data type
data["date"] = pd.to_datetime(data["date"])

# beggining from the 2nd column (idAnio & idMes)
data = data.drop(data.columns[:2], axis=1)

data.head(5)

,TRF_MAESTRO_DESTINATARIOS[cliente_nombre],[VOL],nombre_unificado,date
7264,LA MISION,10.00000,Mascarilla Tiquirurgica 20/5,2023-01-01
7265,MERCADOSCOAHUILENSES,0.89999,Mascarilla Tiquirurgica 20/5,2023-01-01
7266,UNPACO,6.00000,Mascarilla Tiquirurgica 20/5,2023-01-01
7267,ZAPETAS,NaN,Mascarilla Tiquirurgica 20/5,2023-01-01
7268,LA MISION,1.00000,Mascarilla Tiquirurgica 20/5,2023-02-01


**Fechas de corte**

In [16]:
# FILTER TO THE CUT-DATE
data = data[(data["date"] <= "2025-09-01")]

**Layout para los pronósticos**

In [17]:
# layout
data.columns = ["store", "sales", "item", "date"]

# key: store-sku
data["store_item"] = data["store"] + "_" + data["item"]

# final dataset
data = data.iloc[:,[1,3,4]]

# Interpolation

Tabla pivote transpuesta

In [18]:
# transpose table
pivot = pd.pivot_table(data,
                       index='date',
                       values='sales',
                       columns='store_item',
                       aggfunc=np.sum)
pivot.head(5)

/tmp/ipython-input-3786640768.py:2: FutureWarning: The provided callable <function sum at 0x7dc5ba1df880> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = pd.pivot_table(data,


store_item,ALDEKOM_Premier 160 24/4,ALONSO PACHECO GARCIA_Big Roll 400 12/4,ALONSO PACHECO GARCIA_Elite Clasica 180/24,ALONSO PACHECO GARCIA_Elite Cuidado Clasico 287 10/4,ALONSO PACHECO GARCIA_Elite Ultra Absorbente 180 20/1,ALONSO PACHECO GARCIA_Elite Ultra Absorbente 250 20/1,ALONSO PACHECO GARCIA_Ph Big Roll Bp 600Hd 51M Nal 8/6 B,ALONSO PACHECO GARCIA_Premier 400 12/4,ALONSO PACHECO GARCIA_Premier Max 600 10/4,ALONSO PACHECO GARCIA_Premier MegaRoll 400 40/1,...,ZAPETAS_Premier 400 12/4,ZAPETAS_Premier Max 600 10/4,ZAPETAS_Premier Max Fajilla 500 10/4,ZAPETAS_Premier MegaRoll 400 40/1,ZAPETAS_Premier Nova 25X25 12/450,ZAPETAS_Rendiplus 500 6/6,ZAPETAS_Soft Dreams G 4/36,ZAPETAS_Soft Dreams M 4/38,ZAPETAS_Soft Dreams S 4/40,ZAPETAS_Soft Dreams Xg 4/34
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.15120,0.64120,NaN,NaN,0.36525,0.63020,0.00000,0.0,0.0,0.00000
2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.11340,0.32060,NaN,NaN,0.04383,0.94800,28.79999,0.0,0.0,47.59999
2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.60710,1.20545,NaN,NaN,1.46100,0.47400,7.19999,0.0,0.0,6.80000
2023-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.83505,0.00000,NaN,NaN,0.35794,0.71100,0.00000,0.0,0.0,0.00000
2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.06300,0.32060,NaN,NaN,0.00000,1.35564,0.00000,0.0,0.0,2.72000


Método de interpolación

In [19]:
# Linear interpolation
dff_interpolated = pivot.interpolate(method='linear', limit_direction='both')

# drop columns where the sum of values is zero
dff_interpolated = dff_interpolated.loc[:, (dff_interpolated != 0).any(axis=0)]

dff_interpolated.head(5)

store_item,ALONSO PACHECO GARCIA_Big Roll 400 12/4,ALONSO PACHECO GARCIA_Elite Clasica 180/24,ALONSO PACHECO GARCIA_Elite Cuidado Clasico 287 10/4,ALONSO PACHECO GARCIA_Elite Ultra Absorbente 180 20/1,ALONSO PACHECO GARCIA_Elite Ultra Absorbente 250 20/1,ALONSO PACHECO GARCIA_Ph Big Roll Bp 600Hd 51M Nal 8/6 B,ALONSO PACHECO GARCIA_Premier 400 12/4,ALONSO PACHECO GARCIA_Premier Max 600 10/4,ALONSO PACHECO GARCIA_Premier MegaRoll 400 40/1,ALONSO PACHECO GARCIA_Premier Nova 25X25 12/450,...,ZAPETAS_Premier 400 12/4,ZAPETAS_Premier Max 600 10/4,ZAPETAS_Premier Max Fajilla 500 10/4,ZAPETAS_Premier MegaRoll 400 40/1,ZAPETAS_Premier Nova 25X25 12/450,ZAPETAS_Rendiplus 500 6/6,ZAPETAS_Soft Dreams G 4/36,ZAPETAS_Soft Dreams M 4/38,ZAPETAS_Soft Dreams S 4/40,ZAPETAS_Soft Dreams Xg 4/34
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.9918,0.11998,0.08703,0.15753,0.07043,2.778,0.1419,0.26334,0.8016,0.1461,...,12.15120,0.64120,0.0,0.0,0.36525,0.63020,0.00000,0.0,0.0,0.00000
2023-02-01,0.9918,0.11998,0.08703,0.15753,0.07043,2.778,0.1419,0.26334,0.8016,0.1461,...,9.11340,0.32060,0.0,0.0,0.04383,0.94800,28.79999,0.0,0.0,47.59999
2023-03-01,0.9918,0.11998,0.08703,0.15753,0.07043,2.778,0.1419,0.26334,0.8016,0.1461,...,8.60710,1.20545,0.0,0.0,1.46100,0.47400,7.19999,0.0,0.0,6.80000
2023-04-01,0.9918,0.11998,0.08703,0.15753,0.07043,2.778,0.1419,0.26334,0.8016,0.1461,...,6.83505,0.00000,0.0,0.0,0.35794,0.71100,0.00000,0.0,0.0,0.00000
2023-05-01,0.9918,0.11998,0.08703,0.15753,0.07043,2.778,0.1419,0.26334,0.8016,0.1461,...,5.06300,0.32060,0.0,0.0,0.00000,1.35564,0.00000,0.0,0.0,2.72000


Reversion de tabla transpuesta

In [21]:
# revert the transpose table
df = dff_interpolated.reset_index().melt(id_vars=['date'],
                                         var_name='store_item',
                                         value_name='sales')

df.head(3)

,date,store_item,sales
0,2023-01-01,ALONSO PACHECO GARCIA_Big Roll 400 12/4,0.9918
1,2023-02-01,ALONSO PACHECO GARCIA_Big Roll 400 12/4,0.9918
2,2023-03-01,ALONSO PACHECO GARCIA_Big Roll 400 12/4,0.9918


# Dataset for modelling

Layout esperado para Nixtla

In [22]:
# Rename columns to match the Nixtlaverse's expectations

Y_df = df.copy()
#Y_df = Y_df.reset_index()
Y_df = Y_df.rename(columns={
    'store_item': 'unique_id',
    'date': 'ds',
    'sales': 'y'
})

# type of value: target
Y_df['y'] = Y_df['y'].astype(int)

# type of value: id
Y_df['unique_id'] = Y_df['unique_id'].astype(str)

# type of value: date
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

Y_df.tail(5)

,ds,unique_id,y
10060,2025-05-01,ZAPETAS_Soft Dreams Xg 4/34,0
10061,2025-06-01,ZAPETAS_Soft Dreams Xg 4/34,0
10062,2025-07-01,ZAPETAS_Soft Dreams Xg 4/34,0
10063,2025-08-01,ZAPETAS_Soft Dreams Xg 4/34,0
10064,2025-09-01,ZAPETAS_Soft Dreams Xg 4/34,0


# Forecasting methods

## ML Methods with MLForecast

In [ ]:
!pip install window_ops

In [ ]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from mlforecast.utils import PredictionIntervals
from window_ops.expanding import expanding_mean

In [ ]:
# Import the necessary models from various libraries

# LGBMRegressor: A gradient boosting framework that uses tree-based learning algorithms from the LightGBM library
from lightgbm import LGBMRegressor

# XGBRegressor: A gradient boosting regressor model from the XGBoost library
from xgboost import XGBRegressor

# sklearn: Random Forest from the scikit-learn library
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Instantiate the MLForecast object
mlf = MLForecast(models=[LGBMRegressor(max_depth=10),
                         XGBRegressor(max_depth=10, eval_metric='rmse'),
                         RandomForestRegressor()],  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
                 freq='MS',  # Frequency of the data - 'D' for daily frequency
                 #lags=list(range(1, 7)),  # Specific lags to use as regressors: 1 to 6 days
                 lag_transforms = {6:  [expanding_mean], }, # Apply expanding mean transformation to the lag of 1 day
                  date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week', 'dayofyear'],  # Date features to use as regressors
)

In [ ]:
from time import time

# the number of days that the model will use to make a forecast
#window_size = 6*2
horizon = 12

# Split the data into training and testing sets
#train_df = Y_df.groupby('unique_id').head(len(Y_df)//len(Y_df['unique_id'].unique()) - horizon).reset_index(drop=True)
#test_df = Y_df.groupby('unique_id').tail(horizon).reset_index(drop=True)


# Fit the MLForecast models to the data. Removing prediction_intervals to bypass cross-validation issue.
mlf.fit(Y_df, prediction_intervals=PredictionIntervals())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000264 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 322
[LightGBM] [Info] Number of data points in the train set: 7625, number of used features: 7
[LightGBM] [Info] Start training from score 2.489049
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

MLForecast(models=[LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=MS, lag_features=['expanding_mean_lag6'], date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week', 'dayofyear'], num_threads=1)

In [ ]:
# forecast with levels of confidence
#fcst_mlf_df = mlf.predict(horizon, level=[90])

# forecast without levels of confidence
fcst_mlf_df = mlf.predict(horizon)

fcst_mlf_df.tail()

,unique_id,ds,LGBMRegressor,XGBRegressor,RandomForestRegressor
3655,ZAPETAS_Soft Dreams Xg 4/34,2026-05-01,1.473284,1.210121,1.067667
3656,ZAPETAS_Soft Dreams Xg 4/34,2026-06-01,1.291651,0.713354,0.830810
3657,ZAPETAS_Soft Dreams Xg 4/34,2026-07-01,1.353338,7.769153,1.522000
3658,ZAPETAS_Soft Dreams Xg 4/34,2026-08-01,1.399151,7.630795,1.947000
3659,ZAPETAS_Soft Dreams Xg 4/34,2026-09-01,1.232801,7.966363,1.745333


## Statics Methods

In [23]:
# Import necessary models from the statsforecast library
from statsforecast import StatsForecast

from statsforecast.models import (
    # SeasonalNaive: A model that uses the previous season's data as the forecast
    SeasonalNaive,
    # Naive: A simple model that uses the last observed value as the forecast
    Naive,
    # HistoricAverage: This model uses the average of all historical data as the forecast
    HistoricAverage,
    # CrostonOptimized: A model specifically designed for intermittent demand forecasting
    CrostonOptimized,
    # ADIDA: Adaptive combination of Intermittent Demand Approaches, a model designed for intermittent demand
    ADIDA,
    # IMAPA: Intermittent Multiplicative AutoRegressive Average, a model for intermittent series that incorporates autocorrelation
    IMAPA,
    # AutoETS: Automated Exponential Smoothing model that automatically selects the best Exponential Smoothing model based on AIC
    AutoETS
)

In [27]:
# horizon is the number of days in the future for which we will make a forecast
horizon = 12
# the seasonality window is 7 because we have weekly seasonality
season_length = 12
# the number of days that the model will use to make a forecast
#window_size = 6*30
models = [
    SeasonalNaive(season_length=season_length),
    Naive(),
    HistoricAverage(),
    AutoETS(season_length=season_length),
    ADIDA()
]

In [28]:
# Instantiate the StatsForecast class
sf = StatsForecast(
    models=models,  # A list of models to be used for forecasting
    freq='MS',  # The frequency of the time series data (in this case, 'D' stands for daily frequency)
    n_jobs=-1,  # The number of CPU cores to use for parallel execution (-1 means use all available cores)
)

In [29]:
from time import time

# Call the forecast method of the StatsForecast instance to predict the next horizon days

# Level is set to [90], which means that it will compute the 90% prediction interval
#fcst_df = sf.forecast(df=Y_df, h=horizon, level=[90])

fcst_df = sf.forecast(df=Y_df, h=horizon)

fcst_df.tail(5)

,unique_id,ds,SeasonalNaive,Naive,HistoricAverage,AutoETS,ADIDA
3655,ZAPETAS_Soft Dreams Xg 4/34,2026-05-01,0.0,0.0,4.727273,-6.565346,0.554889
3656,ZAPETAS_Soft Dreams Xg 4/34,2026-06-01,0.0,0.0,4.727273,-7.035894,0.554889
3657,ZAPETAS_Soft Dreams Xg 4/34,2026-07-01,0.0,0.0,4.727273,-7.506442,0.554889
3658,ZAPETAS_Soft Dreams Xg 4/34,2026-08-01,0.0,0.0,4.727273,-7.976990,0.554889
3659,ZAPETAS_Soft Dreams Xg 4/34,2026-09-01,0.0,0.0,4.727273,-8.447538,0.554889


# Metrics Erros

Under construcction

# Export

In [ ]:
# merge the forecasting methods
forecast = pd.merge(fcst_df, fcst_mlf_df, on=["unique_id", "ds"], how="left")

# forecast + interpolated dataset
forecast = pd.concat([forecast, Y_df], ignore_index=True).fillna(0)

forecast.head(5)

,unique_id,ds,SeasonalNaive,Naive,HistoricAverage,AutoETS,LGBMRegressor,XGBRegressor,RandomForestRegressor,y
0,ALONSO PACHECO GARCIA_Big Roll 400 12/4,2025-10-01,0.0,0.0,0.0,0.0,0.107768,0.174192,0.133255,0.0
1,ALONSO PACHECO GARCIA_Big Roll 400 12/4,2025-11-01,0.0,0.0,0.0,0.0,0.107768,0.224976,0.227822,0.0
2,ALONSO PACHECO GARCIA_Big Roll 400 12/4,2025-12-01,0.0,0.0,0.0,0.0,0.084324,0.095785,0.103494,0.0
3,ALONSO PACHECO GARCIA_Big Roll 400 12/4,2026-01-01,0.0,0.0,0.0,0.0,0.108070,-0.004724,0.009372,0.0
4,ALONSO PACHECO GARCIA_Big Roll 400 12/4,2026-02-01,0.0,0.0,0.0,0.0,0.109609,0.093414,0.143086,0.0


In [ ]:
# export to directory
forecast.to_excel("/content/drive/Shareddrives/Colaboraciones/FORECAST/forecast_results.xlsx", # direccion / nombre del archivo
                  index=False)